# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [6]:
! conda activate ironhack
!pip install pymongo


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/a7/3c/9f1ad55fbaccf111fc2c3c7309c67035c4dbaed47af92dec3a78d16fe17d/pymongo-4.6.0-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.3/534.3 kB 7.7 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)


In [134]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
client = MongoClient("mongodb://localhost:27017")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [50]:
# Your Code
db = client["test"]
db.list_collection_names()
c = db.get_collection("test")
#c.find_one({},{"_id":0})
list(c.find({"name":"Babelgum"}))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'),
  'name': 'Babelgum',
  'permalink': 'babelgum',
  'crunchbase_url': 'http://www.crunchbase.com/company/babelgum',
  'homepage_url': 'http://babelgum.com',
  'blog_url': 'http://babelgum.com/blog',
  'blog_feed_url': 'http://feeds.feedburner.com/Babelgum',
  'twitter_username': 'Babelgum',
  'category_code': 'games_video',
  'number_of_employees': None,
  'founded_year': 2007,
  'founded_month': 3,
  'founded_day': 1,
  'deadpooled_year': 2013,
  'deadpooled_month': 1,
  'deadpooled_day': 9,
  'deadpooled_url': '',
  'tag_list': 'iptv, web2ireland',
  'alias_list': '',
  'email_address': 'info@babelgum.com',
  'phone_number': '',
  'description': '',
  'created_at': 'Sat Jun 09 08:15:21 UTC 2007',
  'updated_at': 'Wed Oct 16 06:30:25 UTC 2013',
  'overview': '<p>Babelgum is an integrated web and mobile video content platform, free for users and supported by advertising, available on-demand to a global audience.</p>\n\n<p>On March 20th 2009

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [89]:
# Your Code
c.find_one({})
filters = {"number_of_employees":  {"$gt": 5000}}
list(c.find(filters, {"_id":0, "name":1, "number_of_employees":1}).sort("number_of_employees", -1))[:20]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [102]:
# Your Code
c.find_one({})
filters = {"founded_year": {"$gte": 2000,"$lte": 2005}}
list(c.find(filters, {"_id":0, "name":1, "founded_year":1, "ipo":1}))[:20]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004, 'ipo': None},
 {'name': 'Facebook',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Omnidrive', 'founded_year': 2005, 'ipo': None},
 {'name': 'StumbleUpon', 'founded_year': 2002, 'ipo': None},
 {'name': 'Gizmoz', 'founded_year': 2003, 'ipo': None},
 {'name': 'Helio', 'founded_year': 2005, 'ipo': None},
 {'name': 'Plaxo', 'founded_year': 2002, 'ipo': None},
 {'name': 'Technorati', 'founded_year': 2002, 'ipo': None},
 {'name': 'AddThis', 'founded_year': 2004, 'ipo': None},
 {'name': 'Veoh', 'founded_year': 2004, 'ipo': None},
 {'name': 'Jingle Networks', 'founded_year': 2005, 'ipo': None},
 {'name': 'Meetup', 'founded_year': 2002, 'ipo': None},
 {'name': 'LifeLock',
  'founded_year': 2005,
  'ipo': {'valu

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [111]:
# Your Code
#c.find_one({})
filters = {"ipo.valuation_amount":{"$gte":100000000}}
list_df = list(c.find(filters, {"_id":0, "name":1, "ipo.valuation_amount":1}))
pd.DataFrame(list_df)

,name,ipo
0,Facebook,{'valuation_amount': 104000000000.0}
1,Twitter,{'valuation_amount': 18100000000.0}
2,Yelp,{'valuation_amount': 1300000000}
3,LinkedIn,{'valuation_amount': 9310000000.0}
4,Amazon,{'valuation_amount': 100000000000.0}
5,Brightcove,{'valuation_amount': 290000000}
6,KIT digital,{'valuation_amount': 235000000}
7,JumpTV,{'valuation_amount': 100000000}
8,Nielsen,{'valuation_amount': 1600000000}
9,OpenTable,{'valuation_amount': 1050000000}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [131]:
# Your Code
filters = {"$and":[{"number_of_employees":{"$lt":1000}}, {"founded_year":{"$lt":2005}}]}
df = list(c.find(filters, {"_id":0, "name":1,}).sort("number_of_employees", -1))[:20]
pd.DataFrame(df)

,name
0,Infinera Corporation
1,NorthPoint Communications Group
2,888 Holdings
3,Forrester Research
4,SonicWALL
5,Webmetrics
6,Cornerstone OnDemand
7,Yelp
8,MySpace
9,ZoomInfo


### 6. All the companies that don't include the `partners` field.

In [140]:
# Your Code
filters = {"test":{"$ne":"partners"}}
df = list(c.find(filters, {"_id":0, "name":1,}))
pd.DataFrame(df)

,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook
...,...
18796,Adhunk
18797,AfterLogic
18798,goBookmaker
18799,EnteGreat Solutions


### 7. All the companies that have a null type of value on the `category_code` field.

In [143]:
# Your Code
filters = {"category_code":{"$type": "null"}}
df = list(c.find(filters, {"_id":0, "name":1,}))
pd.DataFrame(df)

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing
...,...
2746,Nellix
2747,Cantimer
2748,cruisecritic
2749,Coloroot


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [150]:
# Your Code
# Your Code
filters = {"$and":[{"number_of_employees":{"$gte":100}}, {"number_of_employees":{"$lt":1000}}]}
df = c.find(filters, {"_id":0, "number_of_employees":1})
pd.DataFrame(df)

,number_of_employees
0,600
1,120
2,305
3,644
4,110
...,...
912,500
913,100
914,102
915,385


### 9. Order all the companies by their IPO price in a descending order.

In [156]:
# Your Code
#c.find_one({})
#filters = {}
list_df = list(c.find(filters, {"_id":0, "name":1, "ipo.valuation_amount":1}).sort("ipo.valuation_amount",-1))
pd.DataFrame(list_df)

,name,ipo
0,GREE,{'valuation_amount': 108960000000.0}
1,Facebook,{'valuation_amount': 104000000000.0}
2,Amazon,{'valuation_amount': 100000000000.0}
3,Twitter,{'valuation_amount': 18100000000.0}
4,Groupon,{'valuation_amount': 12800000000.0}
...,...,...
18796,Adhunk,NaN
18797,AfterLogic,NaN
18798,goBookmaker,NaN
18799,EnteGreat Solutions,NaN


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [158]:
# Your Code
filters = {}
list_df = list(c.find(filters, {"_id":0, "name":1, "number_of_employees":1}).sort("number_of_employees",-1))[:20]
pd.DataFrame(list_df)

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [165]:
# Your Code
filters = {"founded_month":{"$in":list(range(6,13))}}
df = list(c.find(filters, {"_id":0, "number_of_employees":1}))
pd.DataFrame(df)

,number_of_employees
0,47.0
1,1600.0
2,60.0
3,NaN
4,NaN
...,...
4032,NaN
4033,NaN
4034,55.0
4035,1.0


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [239]:
# Your Code
filters = {"$and":[{"founded_year":{"$lt":2000}},{"acquisition.price_amount":{"$gt":10000000}}]}
df = list(c.find(filters, {"_id":0, "name":1}))[:10]
pd.DataFrame(df)

,name
0,Postini
1,SideStep
2,Recipezaar
3,PayPal
4,Snapfish
5,Neopets
6,Sun Microsystems
7,Zappos
8,Alibaba
9,Sabre


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [222]:
# Your Code
filters = {"$and":[{"acquisitions.acquired_year":{"$gt":2010}},{"acquisition":{"$ne":None}}]}
df = list(c.find(filters, {"_id":0, "name":1, "acquisition":1}).sort("acquisition.price_amount",1))[20:]
pd.DataFrame(df)

,name,acquisition
0,TeleTech,"{'price_amount': None, 'price_currency_code': ..."
1,TeleTech,"{'price_amount': None, 'price_currency_code': ..."
2,nCircle Network Security,"{'price_amount': None, 'price_currency_code': ..."
3,Vendio,"{'price_amount': None, 'price_currency_code': ..."
4,Brickfish,"{'price_amount': None, 'price_currency_code': ..."
...,...,...
73,Qualcomm Atheros,"{'price_amount': 3100000000.0, 'price_currency..."
74,SuccessFactors,"{'price_amount': 3400000000.0, 'price_currency..."
75,The Weather Channel,"{'price_amount': 3500000000.0, 'price_currency..."
76,Sabre,"{'price_amount': 4300000000.0, 'price_currency..."


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [228]:
# Your Code
filters = {"founded_year":{"$ne":None}}
df = list(c.find(filters, {"_id":0, "name":1, "founded_year":1}))[:20]
pd.DataFrame(df)

,name,founded_year
0,Wetpaint,2005
1,AdventNet,1996
2,Zoho,2005
3,Digg,2004
4,Facebook,2004
5,Omnidrive,2005
6,Postini,1999
7,Geni,2006
8,Fox Interactive Media,1979
9,Twitter,2006


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [237]:
# Your Code
filters = {"founded_day":{"$in":list(range(1,8))}}
df = list(c.find(filters, {"_id":0, "name":1}).sort("acquisition.price_amount",-1))
pd.DataFrame(df)

,name
0,Netscape
1,PayPal
2,Zappos
3,Alibaba
4,Postini
...,...
3216,SEO Sumo
3217,Kicker Films
3218,Getyoo
3219,cotralis


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [238]:
# Your Code
filters = {"$and": [{"category_code":"web"},{"number_of_employees":{"$gt":4000}}]}
df = list(c.find(filters, {"_id":0, "name":1}).sort("number",1))
pd.DataFrame(df)

,name
0,eBay
1,Yahoo!
2,AOL
3,Webkinz
4,Rakuten
5,Expedia
6,Experian
7,Los Angeles Times Media Group
8,Groupon


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [242]:
# Your Code
filters = {"$and":[{"acquisition.price_currency_code":"EUR"},{"acquisition.price_amount":{"$gt":10000000}}]}
df = list(c.find(filters, {"_id":0, "name":1}))[:10]
pd.DataFrame(df)

,name
0,ZYB
1,Apertio
2,Greenfield Online
3,Webedia
4,Wayfinder
5,Tuenti Technologies
6,BioMed Central


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [244]:
# Your Code
filters = {"founded_month":{"$in":list(range(1,4))}}
df = list(c.find(filters, {"_id":0, "name":1, "acquisition.price_amount":1}).sort("acquisition.price_amount",-1))[:10]
pd.DataFrame(df)

,name,acquisition
0,Sun Microsystems,{'price_amount': 7400000000.0}
1,Siebel Systems,{'price_amount': 5850000000.0}
2,YouTube,{'price_amount': 1650000000}
3,Zappos,{'price_amount': 1200000000}
4,Tumblr,{'price_amount': 1100000000}
5,Kiva Systems,{'price_amount': 775000000}
6,Playdom,{'price_amount': 763200000}
7,SonicWALL,{'price_amount': 717000000}
8,Lightera,{'price_amount': 552000000}
9,OptiMedica,{'price_amount': 400000000}


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```